In [194]:
import pandas as pd

In [195]:
df = pd.read_csv("trippub.csv")

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923572 entries, 0 to 923571
Columns: 105 entries, HOUSEID to HBRESDN
dtypes: float64(4), int64(97), object(4)
memory usage: 739.9+ MB


In [197]:
data = pd.DataFrame()

In [198]:
data["ST_WOTAG"] = df["TRAVDAY"]
data["WHYFROM"] = df["WHYFROM"]
data["W_SZ"] = df["STRTTIME"]
data["W_AZ"]= df["ENDTIME"]
data["W_NR"] = df["TDTRPNUM"]
data["wegmin_imp1"] = df["TRVLCMIN"]
data["zweck"] = df["WHYTO"]
data["wegkm"] = df["VMT_MILE"]

In [199]:
testing = data.copy()

In [200]:
testing.head()

,ST_WOTAG,WHYFROM,W_SZ,W_AZ,W_NR,wegmin_imp1,zweck,wegkm
0,2,1,1000,1015,1,15,19,5.244
1,2,19,1510,1530,2,20,1,5.149
2,2,3,700,900,1,120,1,84.004
3,2,1,1800,2030,2,150,3,81.628
4,2,1,845,900,1,15,8,2.250


# Zuordnung der Wochentage anpassen

In [201]:
testing["ST_WOTAG"] = testing["ST_WOTAG"].replace({1:7, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6})

In [202]:
testing["ST_WOTAG"].value_counts()

5    154049
4    150812
3    147798
2    147502
1    137379
6     97536
7     88496
Name: ST_WOTAG, dtype: int64

In [203]:
data["ST_WOTAG"].value_counts()

6    154049
5    150812
4    147798
3    147502
2    137379
7     97536
1     88496
Name: ST_WOTAG, dtype: int64

# Umrechnen der Wergzeiten von hhmm in Minuten

In [204]:
testing["W_SZ"]

0         1000
1         1510
2          700
3         1800
4          845
          ... 
923567     810
923568    1320
923569    1415
923570    1820
923571    1848
Name: W_SZ, Length: 923572, dtype: int64

In [205]:
def to_minutes(hhmm):
    result = 0
    if len(str(hhmm)) == 4:
        hhmm = str(hhmm)
        hh = int(hhmm[0:2])
        mm = int(hhmm[2:4])
    if len(str(hhmm)) ==3:
        hmm = str(hhmm)
        hh = int(hmm[0])
        mm = int(hmm[1:3])
    if len(str(hhmm)) ==2:
        mm = str(hhmm)
        hh = 0
        mm = int(mm)
    if len(str(hhmm)) ==1:
        m = str(hhmm)
        hh = 0
        mm = int(m)
    result = hh*60+mm
    return result

In [206]:
x = 14*60+50
test = to_minutes(1450)
x==test

True

In [207]:
testing["W_SZ_MIN"] = testing["W_SZ"].apply(lambda x: to_minutes(x))

In [208]:
testing["W_AZ_MIN"] = testing["W_AZ"].apply(lambda x: to_minutes(x))

In [209]:
testing.head(10)

,ST_WOTAG,WHYFROM,W_SZ,W_AZ,W_NR,wegmin_imp1,zweck,wegkm,W_SZ_MIN,W_AZ_MIN
0,1,1,1000,1015,1,15,19,5.244,600,615
1,1,19,1510,1530,2,20,1,5.149,910,930
2,1,3,700,900,1,120,1,84.004,420,540
3,1,1,1800,2030,2,150,3,81.628,1080,1230
4,1,1,845,900,1,15,8,2.250,525,540
5,1,8,1430,1445,2,15,1,2.243,870,885
6,4,1,1115,1130,1,15,3,8.017,675,690
7,4,3,2330,2340,2,10,1,8.017,1410,1420
8,4,1,550,605,1,15,16,3.395,350,365
9,4,16,700,715,2,15,1,3.395,420,435


# Neuzuordnung Wegzwecke
-7 (I prefer not to answer) -> 99 (keine Angabe) \
-8 (I don't know) -> 99 (keine Angabe) \
-9 (Not ascertained) -> 99 (keine Angabe) \
01 (Regular home activities (chores, sleep)) -> 8 (nach Hause) \
02 (Work from home(paid)) -> 8 (nach Hause) \
03 (Work) -> 1 (Erreichen des Arbeitsplatzes) \
04 (Work-related meeting / trip) -> 2 (dienstlich/geschäftlich) \
05 (Volunteer acitivities (not paid)) -> 7 (Freizeitaktivität) \
06 (Drop off / pick up someone) -> 6 (Bringen/Holen/Begleiten von Personen) \
07 (Change type of transportation) -> 10 (anderer Zweck) \
08 (Attend school as a student) -> 3 (Erreichen der Ausbildungsstätte/Schule) \
09 Attend child care -> 10 (anderer Zweck) \
10 Attend adult care -> 10 (anderer Zweck) \
11 Buy goods (groceries, clothes, appliances, gas) -> 4 (Einkauf) \
12 Buy services (dry cleaners, banking, service a car, pet care) -> 5 (private Erledigungen) \
13 Buy meals (go out for a meal, snack, carry-out) -> 7 (Freizeitaktivität) \
14 Other general errands (post office, library) -> 5 (private Erledigungen) \
15 Recreational activities (visit parks, movies, bars, museums) -> 7 (Freizeitaktivität) \
16 Exercise (go for a jog, walk, walk the dog, go to the gym) -> 7 (Freizeitaktivität) \
17 Visit friends or relatives -> 7 (Freizeitaktivität) \
18 Health care visit (medical, dental, therapy) -> 5 (private Erledigungen) \
19 Religious or other community activities	-> 7 (Freizeitaktivität) \

## Achtung: 
### Im Ausgangsdatensatz 9 = Rückweg vom vorherigen Weg, kommt hier nicht zum Einsatz

In [210]:
reassignment = {-7:99, -8:99, -9:99, 1:8, 2:8, 3:1, 4:2, 5:7, 6:6, 7:10, 8:3, 9:10, 10:10, 11:4,
                12:5, 13:7, 14:5, 15:7, 16:7, 17:7, 18:5, 19:7}

In [211]:
testing["zweck"] = testing["zweck"].replace(reassignment)

In [212]:
testing["zweck"].value_counts()

8     318777
7     199666
4     132545
1      98968
5      68365
6      56377
3      22133
10     12043
2      11622
97      2844
99       232
Name: zweck, dtype: int64

# Umrechnen Wegelänge

1mile = 1.60934km

In [213]:
testing["wegkm"].apply(lambda x: round(x*1.60934, 3) if x > 0 else -1)

0           8.439
1           8.286
2         135.191
3         131.367
4           3.621
           ...   
923567     -1.000
923568     -1.000
923569     -1.000
923570     -1.000
923571     -1.000
Name: wegkm, Length: 923572, dtype: float64

In [214]:
testing["wegkm"]

0          5.244
1          5.149
2         84.004
3         81.628
4          2.250
           ...  
923567    -1.000
923568    -1.000
923569    -1.000
923570    -1.000
923571    -1.000
Name: wegkm, Length: 923572, dtype: float64

In [215]:
testing.head()

,ST_WOTAG,WHYFROM,W_SZ,W_AZ,W_NR,wegmin_imp1,zweck,wegkm,W_SZ_MIN,W_AZ_MIN
0,1,1,1000,1015,1,15,7,5.244,600,615
1,1,19,1510,1530,2,20,8,5.149,910,930
2,1,3,700,900,1,120,8,84.004,420,540
3,1,1,1800,2030,2,150,1,81.628,1080,1230
4,1,1,845,900,1,15,3,2.250,525,540


In [219]:
testing.to_csv(r'C:\Users\thoma\Desktop\Untitled Folder\NHTS-Analyse\NHTS_trips_processed.csv', index=True, header=True) 